# Demodulación usando redes neuronales

## Librerías

In [1]:
import sofa
import polars as pl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf
import os

# Scikit-Learn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV

# SciPy
from scipy.io import loadmat

# Tensorflow
from tensorflow.keras import models, regularizers, utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import time
%store -r neural_vs_knn_ber18
%store -r neural_vs_knn_ber18_35
%store -r neural_vs_knn_ber18_32
%store -r neural_vs_knn_ber18_30
%store -r neural_vs_knn_ber18_27
%store -r neural_vs_knn_ber18_25
%store -r neural_vs_knn_ber18_23
%store -r neural_vs_knn_ber18_20
%store -r neural_vs_knn_ber18_19
%store -r neural_vs_knn_ber18_18



In [2]:
spacings = ["18", "17.6"]
variables_to_restore = [f"neural_vs_knn_ber{spacing}" for spacing in spacings]

In [3]:
PARAM_GRID_KNN = {"n_neighbors": [3, 5, 7, 9, 11, 13, 15]}
FIGSIZE = (16, 8)


# Función especial para leer todos los datos con la estructura estudiada
def read_data(folder_rx):
    data = {}

    # Leer la carpeta principal
    for folder in os.listdir(folder_rx):
        # Leer las subcarpetas
        if folder.endswith("spacing"):
            data[folder] = {}
            for file in os.listdir(f"{folder_rx}/{folder}"):
                if file.find("consY") != -1:
                    data_name = file.split("_")[2]
                    if data[folder].get(data_name) == None:
                        data[folder][data_name] = {}
                    mat_file_data = loadmat(f"{folder_rx}/{folder}/{file}")
                    data[folder][data_name] = mat_file_data
    return data


# Función para demodular de manera tradicional, usando KNN y ANN
def demodulation(X_rx, X_tx, train_size):
    """
    Realiza la demodulación tradicional, con KNN y ANN

    :param X_rx: Diccionario con los datos recibidos (por espaciamiento)
    :param X_tx: Datos recibidos
    :param train_size: Tamaño entrenamiento
    """
    ber = {}

    for i, snr in enumerate(X_rx):
        # Extraer información
        X_ch_norm = X_rx[snr].get("const_Y").flatten()
        X_ch = sofa.mod_norm(X_ch_norm, 10) * X_ch_norm

        # Arreglos para el BER de cada algoritmo
        trad_ber = np.empty(4)
        knn_ber = np.empty(4)
        neural_ber = np.empty(4)

        for ph in range(4):       
            #Mensaje para verificar avanza (Borrar)
            print(f"----Rotación número {ph+1}. Dato {snr}----\n")
            # Rotar constelación
            rotated_X = X_ch * np.exp(ph * 1j * np.pi / 2)
            # Sincronizar de las señales
            synced_X_tx = sofa.sync_signals(X_tx, rotated_X)
            # Demodular señal transmitida
            y = sofa.demodulate(synced_X_tx, sofa.MOD_DICT)
            #
            trad = sofa.demodulate(rotated_X, sofa.MOD_DICT)
            trad_ber[ph] = sofa.bit_error_rate(trad, y)[0]
            #----------KNN---------------------------
            
            t_inic = time.time()
            # Mejores parámetros para KNN
            best_params_knn = sofa.find_best_params(
                KNeighborsClassifier, PARAM_GRID_KNN, rotated_X, y
            )
            k = best_params_knn["n_neighbors"]
            knn = sofa.demodulate_knn(rotated_X, y, k=k, train_size=train_size)
            knn_ber[ph] = sofa.bit_error_rate(knn, y)[0]

            print(f"BER-KNN {knn_ber[ph]}")
            t_fin = time.time()
            print(f"Tiempo KNN: {t_fin-t_inic}")
            
            #--------------RED NEURONAL------------------------------
            t_ini= time.time()

            max_neurons = 64
            activations = ["relu"]
            layer_props_lst = [
                {"units": max_neurons // (2**i), "activation": activation}
                for i, activation in enumerate(activations)
            ]
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy
            # Demodulación para redes                   
            neural_probs = sofa.demodulate_neural(
                rotated_X,
                y,
                layer_props_lst=layer_props_lst,
                loss_fn=loss_fn,
                train_size=train_size,
            )
            neural = np.argmax(neural_probs, axis=1)
            neural_ber[ph] = sofa.bit_error_rate(neural, y)[0]
            
            print(f"BER red neuronal : {neural_ber[ph]}")
            t_fin = time.time()
            print(f"Tiempo Red Neuronal: {t_fin-t_inic}")
            #-------------------------------------------------------
            # Obtener el índice de la clase con mayor probabilidad de pertenencia
            
            
            # BER de la demodulación con respecto a la transmitida desplazada
            # Índice 0 del retorno para tomar el BER e ignorar la cantidad de errores    

        ber[snr] = {
            "trad": np.amin(trad_ber),
            "knn": np.amin(knn_ber),
            "neural": np.amin(neural_ber),
        }
        print(f"BER para snr = {snr} => {ber[snr]}")
        # Mensaje para saber que se progresa
        print(f"SNR {snr[5:]} terminado.")
    return ber


def curve_fit(f, x, y):
    popt, pcov = sp.optimize.curve_fit(f, x, y)
    return popt


def spacing_ber_eval(ber, spacing):
    """
    Compara resultado de red Neuronal respecto a KNN. Hace gráficas

    :param ber: Diccionario con los datos del ber
    :param spacing: Espaciamiento
    """
    print(
        f"Evaluación de la red neuronal con respecto a KNN para espaciamiento de {spacing} GHz"
    )

    # Lista de strings con los SNR
    SNR = [snr[5:-2] for snr in list(data[f"{spacing}GHz_spacing"].keys())]

    SNR.sort()

    get_ber = lambda algorithm: [
        np.log10(ber.get(f"consY{snr_i}dB").get(algorithm)) for snr_i in SNR
    ]
    #Calcula el ber para cada tipo de demodulación
    tBER = get_ber("trad")
    kBER = get_ber("knn")
    nBER = get_ber("neural")

    # Función para usar en la gráfica
    f = lambda x, a, b, c: a * x**2 + b * x + c

    # Arreglo con los SNR como flotantes
    dSNR = np.array(SNR, dtype=np.float64)
    xSNR = np.linspace(float(SNR[0]), float(SNR[-1]), 1000)

    plt.rcParams["text.usetex"] = True
    plt.figure(figsize=(10, 6))
    # Tradicional: Brown
    # KNN: Green
    # Neural Network: Orange
    plt.scatter(dSNR, tBER, marker="^", c="brown", label="Tradicional")
    plt.plot(
        xSNR,
        f(xSNR, *curve_fit(f, dSNR, tBER)),c="brown",ls="dashed",label="Tradicional (fit)",)
    plt.scatter(dSNR, kBER, marker="x", c="green", label="KNN")
    plt.plot(
        xSNR,
        f(xSNR, *curve_fit(f, dSNR, kBER)),c="green",ls="dashed",label="KNN (fit)",)
    plt.scatter(dSNR, nBER, marker="o", c="orange", label="Neural Network")
    plt.plot(
        xSNR,
        f(xSNR, *curve_fit(f, dSNR, nBER)),
        c="orange",
        ls="dashed",
        label="Neural Network (fit)",
    )

    plt.title(f"{spacing} GHz spacing")
    plt.xlabel("OSNR (dB)")
    yticks = [f"$10^{{{tick}}}$" for tick in plt.yticks()]
    plt.yticks(yticks)
    plt.ylabel("BER")
    plt.legend(loc="upper right")
    plt.grid()
    plt.show()



def calc_once(varname, fn, args):
    """Calcular una variable una sola vez."""
    if varname not in globals():
        return fn(**args)
    return eval(varname)

## Datos experimentales

In [13]:
file_tx = "Datos/2x16QAM_16GBd.mat"
folder_rx = "Datos/"

# Datos transmitidos
X_tx_norm = loadmat(file_tx)
X_tx_norm = X_tx_norm.get("Constellation").flatten()[0][0].flatten()
X_tx = sofa.mod_norm(X_tx_norm, 10)*X_tx_norm

# Leer los datos recibidos
data = read_data(folder_rx)

## Espaciamiento de 18 GHz

In [14]:
neural_vs_knn_ber18_19

{'consY19dB': {'trad': 0.036736202968841856,
  'knn': 0.03423078336272781,
  'neural': 0.03398047104644327}}

In [15]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]
neural_vs_knn_ber18_18 = demodulation(X_rx,X_tx,0.4)
%store neural_vs_knn_ber18_18

----Rotación número 1. Dato consY18dB----

BER-KNN 0.05055619855967078
Tiempo KNN: 22.743672370910645
3402/3402 [==============================] - 4s 1ms/step
BER red neuronal : 0.05051945546737213
Tiempo Red Neuronal: 919.4157259464264
----Rotación número 2. Dato consY18dB----

BER-KNN 0.05065953850676073
Tiempo KNN: 25.86255145072937
3402/3402 [==============================] - 4s 1ms/step
BER red neuronal : 0.050530937683715464
Tiempo Red Neuronal: 503.4716811180115
----Rotación número 3. Dato consY18dB----

BER-KNN 0.05075598912404468
Tiempo KNN: 22.217644214630127
3402/3402 [==============================] - 4s 1ms/step
BER red neuronal : 0.050682502939447385
Tiempo Red Neuronal: 812.160605430603
----Rotación número 4. Dato consY18dB----

BER-KNN 0.050896072163433274
Tiempo KNN: 22.42751979827881
3402/3402 [==============================] - 3s 943us/step
BER red neuronal : 0.05042300485008818
Tiempo Red Neuronal: 842.3141393661499
BER para snr = consY18dB => {'trad': 0.05306161816

In [6]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]
neural_vs_knn_ber18_32 = demodulation(X_rx,X_tx,0.4)
%store neural_vs_knn_ber18_32

----Rotación número 1. Dato consY32dB----

BER-KNN 4.363242210464433e-05
Tiempo KNN: 38.98034381866455
3402/3402 [==============================] - 8s 2ms/step
BER red neuronal : 4.8225308641975306e-05
Tiempo Red Neuronal: 1550.3432927131653
----Rotación número 2. Dato consY32dB----

BER-KNN 3.2150205761316875e-05
Tiempo KNN: 34.137065172195435
3402/3402 [==============================] - 6s 2ms/step
BER red neuronal : 5.281819517930629e-05
Tiempo Red Neuronal: 1018.568172454834
----Rotación número 3. Dato consY32dB----

BER-KNN 4.363242210464433e-05
Tiempo KNN: 36.90018820762634
3402/3402 [==============================] - 9s 3ms/step
BER red neuronal : 4.592886537330982e-05
Tiempo Red Neuronal: 1207.9884250164032
----Rotación número 4. Dato consY32dB----

BER-KNN 3.6743092298647854e-05
Tiempo KNN: 39.573625326156616
3402/3402 [==============================] - 4s 1ms/step
BER red neuronal : 3.2150205761316875e-05
Tiempo Red Neuronal: 1311.5193531513214
BER para snr = consY32dB => {'t

In [5]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]
neural_vs_knn_ber18 = demodulation(X_rx,X_tx,0.4)

----Rotación número 1. Dato consY25dB----

BER-KNN 0.001244672251616696
Tiempo KNN: 27.725266933441162
3402/3402 [==============================] - 4s 1ms/step
BER red neuronal : 0.0013457157554379777
Tiempo Red Neuronal: 770.7269492149353
----Rotación número 2. Dato consY25dB----

BER-KNN 0.0012814153439153438
Tiempo KNN: 29.16118025779724
3402/3402 [==============================] - 5s 1ms/step
BER red neuronal : 0.001267636684303351
Tiempo Red Neuronal: 1030.7945091724396
----Rotación número 3. Dato consY25dB----

BER-KNN 0.001267636684303351
Tiempo KNN: 29.243515968322754
3402/3402 [==============================] - 5s 2ms/step
BER red neuronal : 0.0013020833333333333
Tiempo Red Neuronal: 1207.564539194107
----Rotación número 4. Dato consY25dB----

BER-KNN 0.001290601116990006
Tiempo KNN: 31.08590292930603
3402/3402 [==============================] - 5s 1ms/step
BER red neuronal : 0.0012997868900646678
Tiempo Red Neuronal: 631.5542001724243
BER para snr = consY25dB => {'trad': 0.00

KeyboardInterrupt: 

### 5000 epochs, batch_size 64

In [24]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]
neural_vs_knn_ber18 = demodulation(X_rx,X_tx,0.4)

----Rotación número 1. Dato consY25dB----

BER-KNN 0.0012745260141093474
Tiempo KNN: 62.223000049591064
3402/3402 [==============================] - 8s 2ms/step
BER red neuronal : 0.0015615814226925338
Tiempo Red Neuronal: 3668.5465655326843
----Rotación número 2. Dato consY25dB----

BER-KNN 0.001244672251616696
Tiempo KNN: 42.95404314994812
3402/3402 [==============================] - 7s 2ms/step
BER red neuronal : 0.001357197971781305
Tiempo Red Neuronal: 3321.104240655899
----Rotación número 3. Dato consY25dB----

BER-KNN 0.001272229570840682
Tiempo KNN: 42.479342460632324
3402/3402 [==============================] - 7s 2ms/step
BER red neuronal : 0.0013962375073486185
Tiempo Red Neuronal: 2583.123237133026
----Rotación número 4. Dato consY25dB----

BER-KNN 0.0012217078189300412
Tiempo KNN: 42.39050769805908
3402/3402 [==============================] - 7s 2ms/step
BER red neuronal : 0.0013503086419753086
Tiempo Red Neuronal: 4228.946703910828
BER para snr = consY25dB => {'trad': 0.0

In [30]:
neural_vs_knn_ber18

{'consY40dB': {'trad': 5.741108171663727e-05,
  'knn': 1.8371546149323927e-05,
  'neural': 2.296443268665491e-05}}

In [21]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]

neural_vs_knn_ber18 = calc_once(f"neural_vs_knn_ber{spacing.replace('.', '_')}", 
                                demodulation, {"X_rx": X_rx, "X_tx": X_tx, "train_size": 0.4})

%store neural_vs_knn_ber18

spacing_ber_eval(neural_vs_knn_ber18, spacing)

Stored 'neural_vs_knn_ber18' (dict)
Evaluación de la red neuronal con respecto a KNN para espaciamiento de 18 GHz


AttributeError: 'NoneType' object has no attribute 'get'

In [8]:
spacing = "18"
X_rx = data[f"{spacing}GHz_spacing"]

neural_vs_knn_ber18 = calc_once(f"neural_vs_knn_ber{spacing.replace('.', '_')}", 
                                demodulation, {"X_rx": X_rx, "X_tx": X_tx, "train_size": 0.4})

#%store neural_vs_knn_ber18

spacing_ber_eval(neural_vs_knn_ber18, spacing)

Evaluación de la red neuronal con respecto a KNN para espaciamiento de 18 GHz


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
a = X_rx['consY35dB'].get("const_Y").flatten()

In [ ]:
np.array([a.real, a.imag]).T.shape[1]

2

# Espaciamiento de 17.6 GHz